In [16]:
#Main libraries
import gymnasium as gym
from gymnasium import spaces
from gymnasium.utils import seeding
import cosysairsim as airsim
from cosysairsim import MultirotorClient
import numpy as np
import os
from tqdm import tqdm
import csv
import datetime
from torch.utils.tensorboard import SummaryWriter
#General
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from typing import Dict, Any, Tuple
import logging
from logging import handlers
from copy import deepcopy
from itertools import count
import tqdm
import pickle
from pathlib import Path
import time
#PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import MultivariateNormal
import torch.nn.utils as torch_utils

import multiprocessing
multiprocessing.set_start_method("spawn", force=True)

client = airsim.MultirotorClient()
client.confirmConnection()
print("Connected to AirSim!")

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if GPU is to be used
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)
print(device)

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
from stable_baselines3.common.monitor import Monitor
import numpy as np
from datetime import datetime
import os
import torch
from IPython.display import clear_output
import matplotlib.pyplot as plt
from stable_baselines3.common.callbacks import BaseCallback

Connected!
Client Ver:3 (Min Req: 3), Server Ver:3 (Min Req: 3)

Connected to AirSim!
cuda


In [54]:
client = MultirotorClient()
client.confirmConnection()
client.enableApiControl(True)
client.armDisarm(True)
client.takeoffAsync().join
"""
action=[0,0,1,0]
max_angles = np.radians([30, 30, 45])  # Reduced pitch/roll angles
pitch = np.clip(action[0] * max_angles[0], -max_angles[0], max_angles[0])
roll = np.clip(action[1] * max_angles[1], -max_angles[1], max_angles[1])

# Use exponential mapping for throttle to give more control in hover region
throttle = (action[2])+1 / 2  # Centered around 0.5
yaw_rate = np.clip(action[3] * max_angles[2], -max_angles[2], max_angles[2])

# Add small time delay for stability
client.moveByRollPitchYawrateThrottleAsync(
    roll=roll,
    pitch=pitch,
    yaw_rate=yaw_rate,
    throttle=throttle,
    duration=1000  # Reduced duration for more frequent updates
).join()
"""

Connected!
Client Ver:3 (Min Req: 3), Server Ver:3 (Min Req: 3)



'\naction=[0,0,1,0]\nmax_angles = np.radians([30, 30, 45])  # Reduced pitch/roll angles\npitch = np.clip(action[0] * max_angles[0], -max_angles[0], max_angles[0])\nroll = np.clip(action[1] * max_angles[1], -max_angles[1], max_angles[1])\n\n# Use exponential mapping for throttle to give more control in hover region\nthrottle = (action[2])+1 / 2  # Centered around 0.5\nyaw_rate = np.clip(action[3] * max_angles[2], -max_angles[2], max_angles[2])\n\n# Add small time delay for stability\nclient.moveByRollPitchYawrateThrottleAsync(\n    roll=roll,\n    pitch=pitch,\n    yaw_rate=yaw_rate,\n    throttle=throttle,\n    duration=1000  # Reduced duration for more frequent updates\n).join()\n'

In [26]:
def to_quaternion(self, roll=0.0, pitch=0.0, yaw=0.0):
    """
    Convert roll, pitch, yaw (in radians) to airsim.Quaternionr (x, y, z, w).
    AirSim uses a right-handed coordinate system with X=East, Y=North, Z=Down.
    """
    cy = math.cos(yaw * 0.5)
    sy = math.sin(yaw * 0.5)
    cr = math.cos(roll * 0.5)
    sr = math.sin(roll * 0.5)
    cp = math.cos(pitch * 0.5)
    sp = math.sin(pitch * 0.5)

    w = cy * cr * cp + sy * sr * sp
    x = cy * sr * cp - sy * cr * sp
    y = cy * cr * sp + sy * sr * cp
    z = sy * cr * cp - cy * sr * sp

    return airsim.Quaternionr(x_val=x, y_val=y, z_val=z, w_val=w)


spawn_x = random.uniform(-5, 5)
spawn_y = random.uniform(-5, 5)
spawn_z = random.uniform(-1,1)
spawn_yaw = random.uniform(-math.pi, math.pi)

q = to_quaternion(0, 0, spawn_yaw)
pose = airsim.Pose(
    airsim.Vector3r(spawn_x, spawn_y, spawn_z), q)

client.simSetVehiclePose(pose, True)


In [ ]:
client.moveToPositionAsync(0,0,-10, 0.5).join()

In [22]:
state = client.getMultirotorState()
kinematics = state.kinematics_estimated
pos = kinematics.position
print(pos)

<Vector3r> {   'x_val': 0.0,
    'y_val': 0.0,
    'z_val': 2.055117607116699}


In [ ]:
client.reset()

In [7]:
class CosysAirSimEnv_Basic(gym.Env):
    def __init__(self, training_stage=0):
        super(CosysAirSimEnv_Basic, self).__init__()

        self.client = MultirotorClient()
        self.client.confirmConnection()
        self.client.enableApiControl(True)
        self.client.armDisarm(True)

        self.action_space = spaces.Box(low=-1, high=1, shape=(4,), dtype=np.float32)
        self.observation_space = spaces.Box(low=-5.0, high=5.0, shape=(21,), dtype=np.float32)

        self.max_steps = 200
        self.current_step = 0
        self.episode = 0
        self.wind_factor = 0.0
        self.last_throttle = 0.6
        self.success_count = 0
        
        self.success_threshold = 10
        
        self.training_stage = training_stage
        self.setup_stage_params()
        self.stage_progress = 0
        self.stage_threshold = 25
        
        self.hover_survival_count = 0
        self.consecutive_success = 0
        self.current_target = self._generate_new_target()
        
        self.min_distance = float('inf')
        high = np.array([
            1.0, 1.0, 1.0,      # normalized position
            1.0, 1.0, 1.0,      # normalized velocity
            1.0, 1.0, 1.0,      # normalized angular velocity
            3.0, 3.0, 3.0, 
            1.0, 1.0, 1.0,      # normalized acceleration (in G's)
            1.0, 1.0, 1.0,      # normalized target position
            1.0,                # normalized distance
            1.0,                 # normalized height error
            1.0
        ], dtype=np.float32)
    
        self.observation_space = spaces.Box(low=-high, high=high, dtype=np.float32)
        
        
    def setup_stage_params(self):
        self.params = {
            0: {  # Hovering stage
                'wind': 0.0,
                'area': [10.0, 10.0, 10.0],
                'max_steps': 100,
                'target_type': 'hover',
                'target_radius': 0.5
            },
            1: {  # Near vertical movement
                'wind': 0.0,
                'area': [10.0, 10.0, 10.0],
                'max_steps': 200,
                'target_type': 'vertical',
                'target_radius': 0.5
            },
            2: {  # Close-range horizontal movement
                'wind': 0.0,
                'area': [10.0, 10.0, 5.0],
                'max_steps': 200,
                'target_type': 'horizontal_near',
                'target_radius': 0.3,
                'max_target_dist': 5.0
            },
            3: {  # Medium-range movement
                'wind': 0.0,
                'area': [15.0, 15.0, 8.0],
                'max_steps': 400,
                'target_type': 'free_near',
                'target_radius': 0.3,
                'max_target_dist': 10.0
            },
            4: {  # Long-range movement
                'wind': 0.0,
                'area': [20.0, 20.0, 10.0],
                'max_steps': 500,
                'target_type': 'free',
                'target_radius': 0.3,
                'max_target_dist': 15.0
            },
            5: {  # Extended range movement
                'wind': 0.0,
                'area': [30.0, 30.0, 15.0],  # Expanded area
                'max_steps': 600,
                'target_type': 'free',
                'target_radius': 0.3,
                'max_target_dist': 25.0  # Increased maximum distance
            },
            6: {  # Recovery training
                'wind': 0.0,
                'area': [20.0, 20.0, 10.0],
                'max_steps': 500,
                'target_type': 'free',
                'target_radius': 0.3,
                'max_target_dist': 15.0,
                'recovery_interval': 50  # Apply random action every 50 steps
            },
            7: {  # Moderate wind
                'wind': 0.5,
                'area': [30.0, 30.0, 15.0],
                'max_steps': 600,
                'target_type': 'free',
                'target_radius': 0.5,
                'max_target_dist': 25.0
            },
            8: {  # Strong wind
                'wind': 1.0,
                'area': [30.0, 30.0, 15.0],
                'max_steps': 600,
                'target_type': 'free',
                'target_radius': 0.5,  # Slightly larger radius due to strong wind
                'max_target_dist': 25.0
            }
        }
        self.current_params = self.params[self.training_stage]
        
    def _generate_new_target(self):
        """Internal method to generate a new target based on current parameters"""
        params = self.current_params
        
        if params['target_type'] == 'hover':
            return np.array([0.0, 0.0, -1.0])
            
        elif params['target_type'] == 'vertical':
            height = np.random.uniform(*params['height_range'])
            return np.array([0.0, 0.0, height])
            
        elif params['target_type'] == 'horizontal_near':
            angle = np.random.uniform(0, 2*np.pi)
            dist = np.random.uniform(2.0, params['max_target_dist'])
            return np.array([
                dist * np.cos(angle),
                dist * np.sin(angle),
                -2.0
            ])
            
        elif params['target_type'] in ['free_near', 'free']:
            while True:
                point = np.random.uniform(
                    low=[-params['max_target_dist'], -params['max_target_dist'], -4.0],
                    high=[params['max_target_dist'], params['max_target_dist'], -1.0],
                    size=(3,)
                )
                if np.linalg.norm(point - self._get_current_position()) > 2.0:
                    return point

    def _get_target(self):
        """Return the current target"""
        return self.current_target
    
    def _get_distance_target(self):
        return np.linalg.norm(self._get_current_position() - self._get_target())
        
    def _get_current_position(self):
        state = self.client.getMultirotorState()
        kinematics = state.kinematics_estimated
        pos = kinematics.position

        return np.array([pos.x_val, pos.y_val, pos.z_val], dtype=np.float32)

    def _get_current_velocity(self):
        state = self.client.getMultirotorState()
        kinematics = state.kinematics_estimated
        vel = kinematics.linear_velocity

        return np.array([vel.x_val, vel.y_val, vel.z_val], dtype=np.float32)
    
    def _get_imu_data(self):
        imu_data = self.client.getImuData()
        # IMU typically gives angular velocity and linear acceleration
        # We'll store them as [wx, wy, wz, ax, ay, az]
        ang_vel = imu_data.angular_velocity
        lin_acc = imu_data.linear_acceleration
        return np.array([
            ang_vel.x_val, ang_vel.y_val, ang_vel.z_val,
            lin_acc.x_val, lin_acc.y_val, lin_acc.z_val
        ], dtype=np.float32)

    def _get_mag_data(self):
        mag_data = self.client.getMagnetometerData()
        # magnetometer.x, .y, .z
        return np.array([
            mag_data.magnetic_field_body.x_val,
            mag_data.magnetic_field_body.y_val,
            mag_data.magnetic_field_body.z_val
        ], dtype=np.float32)

    def _get_gps_data(self):
        gps_data = self.client.getGpsData()
        # gps_data.gnss.geo_point.latitude, .longitude, .altitude
        return np.array([
            gps_data.gnss.geo_point.latitude,
            gps_data.gnss.geo_point.longitude,
            gps_data.gnss.geo_point.altitude
        ], dtype=np.float32)

    def _get_baro_data(self):
        baro_data = self.client.getBarometerData()
        # baro_data.altitude, baro_data.pressure, baro_data.qnh
        # We'll just use altitude
        return np.array([baro_data.altitude], dtype=np.float32)

    def _get_observation(self):
        """
        #Get and standardize observations.
        position = self._get_current_position()
        velocity = self._get_current_velocity()
        imu_data = self._get_imu_data()
        target = self._get_target()
        distance = self._get_distance_target()

        # We'll ignore GPS and magnetometer data as they're less relevant for this task
        # and could add noise to the learning process

        # Standardize position: Scale by area bounds
        normalized_pos = position / np.array(self.current_params['area'])

        # Standardize velocity: Most drones operate within -10 to 10 m/s range
        normalized_vel = np.clip(velocity / 10.0, -1.0, 1.0)

        # Standardize IMU data
        angular_vel = imu_data[:3] / 10.0  # Angular velocity (rad/s)
        linear_acc = imu_data[3:6] / 9.81  # Linear acceleration (normalize by g)

        # Standardize target and distance
        normalized_target = target / np.array(self.current_params['area'])
        normalized_distance = distance / np.linalg.norm(self.current_params['area'])

        # Height error (important for hovering)
        height_error = (position[2] - target[2]) / self.current_params['area'][2]

        # Combine all observations
        observation = np.concatenate([
            normalized_pos,        # [0:3]   - Normalized position (x, y, z)
            normalized_vel,        # [3:6]   - Normalized velocity (vx, vy, vz)
            angular_vel,          # [6:9]   - Normalized angular velocity (wx, wy, wz)
            linear_acc,          # [9:12]  - Normalized linear acceleration (ax, ay, az)
            normalized_target,    # [12:15] - Normalized target position
            [normalized_distance],# [15]    - Normalized distance to target
            [height_error]       # [16]    - Normalized height error
        ])
        # Clip all values to ensure they stay within a reasonable range
        observation = np.clip(observation, -5.0, 5.0)
        
        return observation.astype(np.float32)"""
        position = self._get_current_position()
        velocity = self._get_current_velocity()
        imu_data = self._get_imu_data()
        target = self._get_target()
        distance = self._get_distance_target()

        # Normalize position relative to the current stage's boundaries
        area_bounds = np.array(self.current_params['area'])
        normalized_pos = np.clip(position / area_bounds, -1.0, 1.0)
        
        # Normalize velocity (typical drone speeds rarely exceed ±10 m/s)
        velocity_scale = np.array([10.0, 10.0, 5.0])  # Less vertical velocity range
        normalized_vel = np.clip(velocity / velocity_scale, -1.0, 1.0)
        
        # Normalize IMU data
        # Angular velocity (typically ±5 rad/s)
        angular_vel = np.clip(imu_data[:3] / 5.0, -1.0, 1.0)
        # Linear acceleration (normalize by g ≈ 9.81 m/s²)
        linear_acc = np.clip(imu_data[3:6] / 9.81, -3.0, 3.0)  # Allow for higher G-forces
        
        # Normalize target position
        normalized_target = np.clip(target / area_bounds, -1.0, 1.0)
        
        # Normalize distance relative to maximum possible distance in current area
        max_possible_distance = np.linalg.norm(area_bounds)
        normalized_distance = np.clip(distance / max_possible_distance, 0.0, 1.0)
        
        # Height error normalization
        max_height_error = area_bounds[2]
        height_error = (position[2] - target[2]) / max_height_error
        normalized_height_error = np.clip(height_error, -1.0, 1.0)

        relative_pos = (target - position) / np.array(self.current_params['area'])
        
        target_direction = (target - position) / (np.linalg.norm(target - position) + 1e-6)
        relative_vel = np.dot(velocity, target_direction)
        
        observation = np.concatenate([
            normalized_pos,        # [0:3]
            normalized_vel,        # [3:6]
            angular_vel,          # [6:9]
            linear_acc,          # [9:12]
            relative_pos,
            normalized_target,        # [12:15]
            [normalized_distance],# [15]
            [normalized_height_error],      # [16]
            [relative_vel]       # [17] - Added velocity towards target
        ])
        return observation.astype(np.float32)

    def _apply_wind(self):
        """Applies a simulated wind force to the drone."""
        wind_x = np.random.uniform(-self.wind_factor, self.wind_factor)
        wind_y = np.random.uniform(-self.wind_factor, self.wind_factor)
        wind_z = np.random.uniform(-self.wind_factor / 2, self.wind_factor / 2)
        self.client.simSetWind(airsim.Vector3r(wind_x, wind_y, wind_z))
    
    def reset(self, seed=None, options=None):
        if seed is not None:
            self.seed(seed)

        # Reset the drone
        self.client.reset()
        self.client.enableApiControl(True)
        self.client.armDisarm(True)
        self.client.takeoffAsync().join()

        self.current_step = 0
        self.consecutive_success = 0
        self.episode += 1
        self._apply_wind()
        
        if not hasattr(self, 'current_target') or self.current_target is None:
            self._get_target()  # This will generate a new target
        
        observation = self._get_observation()
        return observation, {}

    def step(self, action):
        self.current_step += 1
        action = np.clip(action, -1.0, 1.0)
        if self.training_stage == 6:
            if self.current_step % self.current_params['recovery_interval'] == 0:
                # Apply random action
                random_action = np.random.uniform(-1, 1, size=4)
                self.recovery_action_timer = 10  # Recovery period
                action = random_action
            elif self.recovery_action_timer > 0:
                self.recovery_action_timer -= 1
        # Process controls - separate throttle handling
        max_angles = np.radians([45, 45, 45])  # pitch, roll, yaw_rate
        
        pitch = np.clip(action[0] * max_angles[0], -max_angles[0], max_angles[0])
        roll = np.clip(action[1] * max_angles[1], -max_angles[1], max_angles[1])
        throttle = (action[2] + 1.0) / 2.0  # Convert from [-1,1] to [0,1]
        yaw_rate = np.clip(action[3] * max_angles[2], -max_angles[2], max_angles[2])
        
        # Execute action in AirSim
        self.client.moveByRollPitchYawrateThrottleAsync(
            roll=roll,
            pitch=pitch,
            yaw_rate=yaw_rate,
            throttle=throttle,
            duration=0.25
        ).join()
        
        self._apply_wind()
        observation = self._get_observation()
        reward, terminated = self._calculate_reward()
        
        truncated = self.current_step >= self.current_params['max_steps']
        #if truncated:
        #    reward += 10
        """
        if self.training_stage == 0:
            # If we ended the episode by hitting max_steps (truncate) WITHOUT crashing (terminated=False),
            # then we survived. Increment the counter.
            if truncated:
                self.hover_survival_count += 1
                reward += 50

                # If we've survived enough times in a row, move to stage 1
                if self.hover_survival_count >= self.num_consecutive_survivals_needed:
                    self.training_stage = 1
                    self.setup_stage_params()
                    self.hover_survival_count = 0
                    print(
                        f"=== Moved to stage {self.training_stage} after "
                        f"{self.num_consecutive_survivals_needed} consecutive survival episodes. ==="
                    )
        """
        return observation, reward, terminated, truncated, {}
    def _calculate_reward(self):
        pos = self._get_current_position()
        vel = self._get_current_velocity()
        angular_vel = self._get_imu_data()[:3]
        target = self._get_target()
        distance = np.linalg.norm(pos - target)
        reward_scale = 0.005
        reward = 0.0
        # Immediate failure conditions (keep these)
        if self.client.simGetCollisionInfo().has_collided:
            reward -= 200*reward_scale
            return reward, True
        if any(abs(p) > a for p, a in zip(pos, self.current_params['area'])):
            reward -= 100*reward_scale
            return reward, True
        # Main distance reward - smoother gradient
        distance_reward = -distance+1  # Linear scaling for more consistent gradient
        
        """
        # Stability penalty - much gentler
        stability_penalty = (
            0.05 * np.linalg.norm(angular_vel) +  # Reduced rotation penalty
            0.02 * np.linalg.norm(vel)            # Reduced velocity penalty
        )
        reward -= stability_penalty
        """
        if self.training_stage == 0:  # Hovering
            # Pure hovering - focus on stability
            vertical_distance = abs(pos[2] - target[2])
            reward += (-vertical_distance + 1.0)*reward_scale
            reward += (distance_reward*0.5)*reward_scale
            
            
        elif self.training_stage == 1:  # Vertical movement
            # Reward vertical progress toward target
            vertical_distance = abs(pos[2] - target[2])
            reward += (-vertical_distance + 1.0)*reward_scale
            reward += (distance_reward*0.5)*reward_scale
            # Penalize horizontal drift more strongly
            horizontal_drift = np.linalg.norm(pos[:2] - target[:2])
            reward -= (0.5 * horizontal_drift)*reward_scale
            
            reward += (distance_reward*0.5)*reward_scale
            
        elif self.training_stage == 2:  # Close-range horizontal
            # Reward horizontal progress
            if hasattr(self, '_prev_distance'):
                progress = self._prev_distance - distance
                reward += (3.0 * progress)*reward_scale  # Stronger reward for deliberate movement
            self._prev_distance = distance
            
            # Height stability becomes secondary but still important
            reward += (distance_reward)*reward_scale
            
        elif self.training_stage >= 3:  # Medium-range movement
            # Balance distance and control
            if hasattr(self, '_prev_distance'):
                progress = self._prev_distance - distance
                reward += (2.0 * progress)*reward_scale
            self._prev_distance = distance
            reward += (distance_reward)*reward_scale
        # Success condition
        success_radius = self.current_params['target_radius']
        if distance < success_radius:
            bonus = 0
            bonus += (25.0)*reward_scale
            self.consecutive_success += 1
            
            if self.consecutive_success % self.success_threshold == 0:
                self.current_target = self._generate_new_target()
                bonus += 50.0*reward_scale  
                self.success_count += 1
        
            if self.success_count >= self.stage_threshold:
                self.training_stage = min(8, self.training_stage + 1)
                print(f"\n=== Advanced to stage {self.training_stage} ===")
                self.success_count = 0
                self.setup_stage_params()
                
            return reward, True
        return reward, False

    def close(self):
        self.client.reset()
        self.client.enableApiControl(False)
        self.client.armDisarm(False)

In [56]:
class CosysAirSimEnv_Velocity(gym.Env):
    def __init__(self, training_stage=5):
        super(CosysAirSimEnv_Velocity, self).__init__()

        self.client = MultirotorClient()
        self.client.confirmConnection()
        self.client.enableApiControl(True)
        self.client.armDisarm(True)

        self.action_space = spaces.Box(low=-1, high=1, shape=(4,), dtype=np.float32)
        self.observation_space = spaces.Box(low=-5.0, high=5.0, shape=(24,), dtype=np.float32)

        self.max_steps = 200
        self.current_step = 0
        self.episode = 0
        self.wind_factor = 0.0
        self.last_throttle = 0.6
        self.success_count = 0

        self.success_threshold = 20
        
        self.training_stage = training_stage
        self.setup_stage_params()
        self.stage_progress = 0
        self.stage_threshold = 50
        
        self.hover_survival_count = 0
        self.consecutive_success = 0
        self.current_target = self._generate_new_target()
        
        self.episode_distances = []
        self.recovery_action_timer = 0
        
        high = np.array([
            1.0, 1.0, 1.0,      # normalized position
            1.0, 1.0, 1.0,      # normalized velocity
            1.0, 1.0, 1.0,      # normalized angular velocity
            3.0, 3.0, 3.0, 
            1.0, 1.0, 1.0,      # normalized acceleration (in G's)
            1.0, 1.0, 1.0,      # normalized target position
            1.0,                # normalized distance
            1.0,                 # normalized height error
            1.0,
            1.0,
            1.0, 1.0
        ], dtype=np.float32)
    
        self.observation_space = spaces.Box(low=-high, high=high, dtype=np.float32)
        self.episode_min_distance = 1
        
    def setup_stage_params(self):
        self.params = {
            0: {  # Hovering stage
                'wind': 0.0,
                'area': [10.0, 10.0, 10.0],
                'max_steps': 100,
                'target_type': 'hover',
                'target_radius': 0.2,
                'spawn_range': 1.0
            },
            1: {  # Near vertical movement
                'wind': 0.0,
                'area': [10.0, 10.0, 10.0],
                'max_steps': 100,
                'target_type': 'vertical',
                'target_radius': 0.2,
                'height_range': [-6.0, -1.0],
                'spawn_range': 2.0
            },
            2: {  # Short-range horizontal movement
                'wind': 0.0,
                'area': [10.0, 10.0, 5.0],
                'max_steps': 150,
                'target_type': 'horizontal_near',
                'target_radius': 0.2,
                'max_target_dist': 2.0,
                'spawn_range': 2.0
            },
            3: {  # Long-range horizontal movement
                'wind': 0.0,
                'area': [10.0, 10.0, 5.0],
                'max_steps': 200,
                'target_type': 'horizontal_near',
                'target_radius': 0.2,
                'max_target_dist': 6.0,
                'spawn_range': 2.0
            },
            4: {  # Medium-range movement
                'wind': 0.0,
                'area': [15.0, 15.0, 8.0],
                'max_steps': 200,
                'target_type': 'free_near',
                'target_radius': 0.2,
                'max_target_dist': 6.0,
                'spawn_range': 3.0
            },
            5: {  # Medium-range movement
                'wind': 0.0,
                'area': [15.0, 15.0, 8.0],
                'max_steps': 250,
                'target_type': 'free_near',
                'target_radius': 0.25,
                'max_target_dist': 8.0,
                'spawn_range': 3.0
            },
            6: {  # Long-range movement
                'wind': 0.0,
                'area': [20.0, 20.0, 10.0],
                'max_steps': 300,
                'target_type': 'free',
                'target_radius': 0.25,
                'max_target_dist': 15.0,
                'spawn_range': 3.0
            },
            7: {  # Recovery training
                'wind': 0.0,
                'area': [20.0, 20.0, 10.0],
                'max_steps': 300,
                'target_type': 'free',
                'target_radius': 0.25,
                'max_target_dist': 15.0,
                'recovery_interval': 50, # Apply random action every 50 steps
                'spawn_range': 4.0  
            },
            8: {  # Moderate wind
                'wind': 1.0,
                'area': [30.0, 30.0, 15.0],
                'max_steps': 300,
                'target_type': 'free',
                'target_radius': 0.25,
                'max_target_dist': 15.0,
                'recovery_interval': 150,
                'spawn_range': 4.0
            },
            9: {  # Strong wind
                'wind': 5.0,
                'area': [30.0, 30.0, 15.0],
                'max_steps': 300,
                'target_type': 'free',
                'target_radius': 0.25,  # Slightly larger radius due to strong wind
                'max_target_dist': 15.0,
                'recovery_interval': 150,
                'spawn_range': 4.0
            }
        }
        self.current_params = self.params[self.training_stage]
        
    def _generate_new_target(self):
        """Internal method to generate a new target based on current parameters"""
        params = self.current_params
        
        if params['target_type'] == 'hover':
            return np.array([0.0, 0.0, -1.75])
            
        elif params['target_type'] == 'vertical':
            height = np.random.uniform(*params['height_range'])
            return np.array([0.0, 0.0, height])
            
        elif params['target_type'] == 'horizontal_near':
            angle = np.random.uniform(0, 2*np.pi)
            dist = np.random.uniform(4.0, params['max_target_dist'])
            z_val = np.random.uniform(-1.0, -2.0)
            return np.array([
                dist * np.cos(angle),
                dist * np.sin(angle),
                z_val
            ])
            
        elif params['target_type'] in ['free_near', 'free']:
            while True:
                point = np.random.uniform(
                    low=[-params['max_target_dist'], -params['max_target_dist'], -5.0],
                    high=[params['max_target_dist'], params['max_target_dist'], -1.0],
                    size=(3,)
                )
                if np.linalg.norm(point - self._get_current_position()) > 2.0:
                    return point

    def _get_target(self):
        """Return the current target"""
        return self.current_target
    
    def _get_distance_target(self):
        return np.linalg.norm(self._get_current_position() - self._get_target())
        
    def _get_current_position(self):
        state = self.client.getMultirotorState()
        kinematics = state.kinematics_estimated
        pos = kinematics.position

        return np.array([pos.x_val, pos.y_val, pos.z_val], dtype=np.float32)

    def _get_current_velocity(self):
        state = self.client.getMultirotorState()
        kinematics = state.kinematics_estimated
        vel = kinematics.linear_velocity

        return np.array([vel.x_val, vel.y_val, vel.z_val], dtype=np.float32)
    
    def _get_imu_data(self):
        imu_data = self.client.getImuData()
        # IMU typically gives angular velocity and linear acceleration
        # We'll store them as [wx, wy, wz, ax, ay, az]
        ang_vel = imu_data.angular_velocity
        lin_acc = imu_data.linear_acceleration
        return np.array([
            ang_vel.x_val, ang_vel.y_val, ang_vel.z_val,
            lin_acc.x_val, lin_acc.y_val, lin_acc.z_val
        ], dtype=np.float32)

    def _get_mag_data(self):
        mag_data = self.client.getMagnetometerData()
        # magnetometer.x, .y, .z
        return np.array([
            mag_data.magnetic_field_body.x_val,
            mag_data.magnetic_field_body.y_val,
            mag_data.magnetic_field_body.z_val
        ], dtype=np.float32)

    def _get_gps_data(self):
        gps_data = self.client.getGpsData()
        # gps_data.gnss.geo_point.latitude, .longitude, .altitude
        return np.array([
            gps_data.gnss.geo_point.latitude,
            gps_data.gnss.geo_point.longitude,
            gps_data.gnss.geo_point.altitude
        ], dtype=np.float32)

    def _get_baro_data(self):
        baro_data = self.client.getBarometerData()
        # baro_data.altitude, baro_data.pressure, baro_data.qnh
        # We'll just use altitude
        return np.array([baro_data.altitude], dtype=np.float32)

    def _get_lidar_data(self):
        """
        Retrieves LiDAR data from the 'LidarFront' sensor and returns a single float:
        the minimum distance to an obstacle in front of the drone (or the max range if no hits).
        """
        lidar_data = self.client.getLidarData(lidar_name="LidarFront")
        
        # If no data or sensor not active
        if not lidar_data.point_cloud or len(lidar_data.point_cloud) < 3:
            # No points -> assume no obstacles within range
            return np.array([1.0], dtype=np.float32)  # normalized '1.0' as 'no obstacle in range'
        
        points = np.array(lidar_data.point_cloud, dtype=np.float32).reshape(-1, 3)

        distances = np.linalg.norm(points, axis=1)
        min_distance = np.min(distances)

        max_range = 4.0
        clipped_dist = np.clip(min_distance, 0.0, max_range)
        
        # Normalize from 0..20 -> 0..1
        #  (0 means extremely close obstacle, 1 means no obstacle within range)
        normalized_dist = clipped_dist / max_range
        return np.array([normalized_dist], dtype=np.float32)

    def quaternion_to_euler(self, q):
        """
        Convert AirSim Quaternionr (q.w_val, q.x_val, q.y_val, q.z_val)
        to Euler angles (roll, pitch, yaw) in radians.
        
        AirSim uses a right-handed coordinate system:
        - X axis: East
        - Y axis: North
        - Z axis: Down
        """
        w = q.w_val
        x = q.x_val
        y = q.y_val
        z = q.z_val

        # Pre-calculate repeated values
        ysqr = y * y

        # Roll (x-axis rotation)
        t0 = +2.0 * (w * x + y * z)
        t1 = +1.0 - 2.0 * (x * x + ysqr)
        roll = math.atan2(t0, t1)

        # Pitch (y-axis rotation)
        t2 = +2.0 * (w * y - z * x)
        t2 = 1.0 if t2 > 1.0 else t2
        t2 = -1.0 if t2 < -1.0 else t2
        pitch = math.asin(t2)

        # Yaw (z-axis rotation)
        t3 = +2.0 * (w * z + x * y)
        t4 = +1.0 - 2.0 * (ysqr + z * z)
        yaw = math.atan2(t3, t4)

        return roll, pitch, yaw

    def to_quaternion(self, roll=0.0, pitch=0.0, yaw=0.0):
        """
        Convert roll, pitch, yaw (in radians) to airsim.Quaternionr (x, y, z, w).
        AirSim uses a right-handed coordinate system with X=East, Y=North, Z=Down.
        """
        cy = math.cos(yaw * 0.5)
        sy = math.sin(yaw * 0.5)
        cr = math.cos(roll * 0.5)
        sr = math.sin(roll * 0.5)
        cp = math.cos(pitch * 0.5)
        sp = math.sin(pitch * 0.5)

        w = cy * cr * cp + sy * sr * sp
        x = cy * sr * cp - sy * cr * sp
        y = cy * cr * sp + sy * sr * cp
        z = sy * cr * cp - cy * sr * sp

        return airsim.Quaternionr(x_val=x, y_val=y, z_val=z, w_val=w)

    def _get_observation(self):
        """
        #Get and standardize observations.
        position = self._get_current_position()
        velocity = self._get_current_velocity()
        imu_data = self._get_imu_data()
        target = self._get_target()
        distance = self._get_distance_target()

        # We'll ignore GPS and magnetometer data as they're less relevant for this task
        # and could add noise to the learning process

        # Standardize position: Scale by area bounds
        normalized_pos = position / np.array(self.current_params['area'])

        # Standardize velocity: Most drones operate within -10 to 10 m/s range
        normalized_vel = np.clip(velocity / 10.0, -1.0, 1.0)

        # Standardize IMU data
        angular_vel = imu_data[:3] / 10.0  # Angular velocity (rad/s)
        linear_acc = imu_data[3:6] / 9.81  # Linear acceleration (normalize by g)

        # Standardize target and distance
        normalized_target = target / np.array(self.current_params['area'])
        normalized_distance = distance / np.linalg.norm(self.current_params['area'])

        # Height error (important for hovering)
        height_error = (position[2] - target[2]) / self.current_params['area'][2]

        # Combine all observations
        observation = np.concatenate([
            normalized_pos,        # [0:3]   - Normalized position (x, y, z)
            normalized_vel,        # [3:6]   - Normalized velocity (vx, vy, vz)
            angular_vel,          # [6:9]   - Normalized angular velocity (wx, wy, wz)
            linear_acc,          # [9:12]  - Normalized linear acceleration (ax, ay, az)
            normalized_target,    # [12:15] - Normalized target position
            [normalized_distance],# [15]    - Normalized distance to target
            [height_error]       # [16]    - Normalized height error
        ])
        # Clip all values to ensure they stay within a reasonable range
        observation = np.clip(observation, -5.0, 5.0)
        
        return observation.astype(np.float32)"""
        position = self._get_current_position()
        velocity = self._get_current_velocity()
        imu_data = self._get_imu_data()
        target = self._get_target()
        distance = self._get_distance_target()

        # Normalize position relative to the current stage's boundaries
        area_bounds = np.array(self.current_params['area'])
        normalized_pos = np.clip(position / area_bounds, -1.0, 1.0)
        
        # Normalize velocity (typical drone speeds rarely exceed ±10 m/s)
        velocity_scale = np.array([10.0, 10.0, 5.0])  # Less vertical velocity range
        normalized_vel = np.clip(velocity / velocity_scale, -1.0, 1.0)
        
        # Normalize IMU data
        # Angular velocity (typically ±5 rad/s)
        angular_vel = np.clip(imu_data[:3] / 5.0, -1.0, 1.0)
        # Linear acceleration (normalize by g ≈ 9.81 m/s²)
        linear_acc = np.clip(imu_data[3:6] / 9.81, -3.0, 3.0)  # Allow for higher G-forces
        
        # Normalize target position
        normalized_target = np.clip(target / area_bounds, -1.0, 1.0)
        
        # Normalize distance relative to maximum possible distance in current area
        max_possible_distance = np.linalg.norm(area_bounds)
        normalized_distance = np.clip(distance / max_possible_distance, 0.0, 1.0)
        
        # Height error normalization
        max_height_error = area_bounds[2]
        height_error = (position[2] - target[2]) / max_height_error
        normalized_height_error = np.clip(height_error, -1.0, 1.0)

        relative_pos = (target - position) / np.array(self.current_params['area'])
        
        state = self.client.getMultirotorState()
        orientation = state.kinematics_estimated.orientation
        roll, pitch, yaw = self.quaternion_to_euler(orientation)
        yaw_sin = np.sin(yaw)
        yaw_cos = np.cos(yaw)
        
        target_direction = (target - position) / (np.linalg.norm(target - position) + 1e-6)
        relative_vel = np.dot(velocity, target_direction)
        lidar_front = self._get_lidar_data()
        observation = np.concatenate([
            normalized_pos,        # [0:3]
            normalized_vel,        # [3:6]
            angular_vel,          # [6:9]
            linear_acc,          # [9:12]
            relative_pos,       # [12:15]
            normalized_target,      # [15:18]
            [normalized_distance],# [18]
            [normalized_height_error],      # [19]
            [relative_vel],       # [20] - Added velocity towards target
            lidar_front,           # [21]
            [yaw_sin, yaw_cos]      # [22:23]
        ])
        return observation.astype(np.float32)

    def _apply_wind(self):
        """Applies a simulated wind force to the drone."""
        wind_x = np.random.uniform(-self.wind_factor, self.wind_factor)
        wind_y = np.random.uniform(-self.wind_factor, self.wind_factor)
        wind_z = np.random.uniform(-self.wind_factor / 2, self.wind_factor / 2)
        self.client.simSetWind(airsim.Vector3r(wind_x, wind_y, wind_z))
    
    def reset(self, seed=None, options=None):
        if seed is not None:
            self.seed(seed)

        # Reset the drone
        self.client.reset()
        self.client.enableApiControl(True)
        self.client.armDisarm(True)
        
        params = self.current_params
        spawn_x = random.uniform(-params['spawn_range'], params['spawn_range'])
        spawn_y = random.uniform(-params['spawn_range'], params['spawn_range'])
        spawn_z = random.uniform(-1,1)
        spawn_yaw = random.uniform(-math.pi, math.pi)
        
        q = self.to_quaternion(0, 0, spawn_yaw)
        pose = airsim.Pose(
            airsim.Vector3r(spawn_x, spawn_y, spawn_z), q)

        self.client.simSetVehiclePose(pose, True)
        
        self.client.takeoffAsync().join()
        #self.client.moveToPositionAsync(spawn_x,spawn_y,spawn_z, 5).join()
        #self.client.rotateToYawAsync(spawn_yaw).join()

        self.current_step = 0
        self.consecutive_success = 0
        self.episode += 1
        self._apply_wind()
        
        self.episode_distances = []
        """
        if not hasattr(self, 'current_target') or self.current_target is None:
            self._get_target()  # This will generate a new target
        """
        self._generate_new_target()
        observation = self._get_observation()
        return observation, {}

    def step(self, action):
        self.current_step += 1
        action = np.clip(action, -1.0, 1.0)
        if self.training_stage >= 7:
                # Apply random action
            if self.recovery_action_timer > 0:
                self.recovery_action_timer -= 1
            else:
                if np.random.rand() <= 1/self.current_params['recovery_interval']:
                    random_action = np.random.uniform(-1, 1, size=4)
                    self.recovery_action_timer = 10  # Recovery period
                    action = random_action
        # Process controls - separate throttle handling
        max_vx, max_vy, max_vz = 5.0, 5.0, 5.0   # [m/s]
        max_yaw_deg = 45.0                      # [deg/s], or you can do rad/s

        vx = float(action[0]) * max_vx
        vy = float(action[1]) * max_vy
        vz = float(action[2]) * max_vz
        yaw_rate_deg = float(action[3]) * max_yaw_deg
        
        yaw_mode = airsim.YawMode(is_rate=True, yaw_or_rate=yaw_rate_deg)

        # Move by velocity in BODY frame
        self.client.moveByVelocityBodyFrameAsync(
            vx=vx,
            vy=vy,
            vz=vz,
            duration=0.25,  # small step
            drivetrain=airsim.DrivetrainType.MaxDegreeOfFreedom,
            yaw_mode=yaw_mode
        ).join()
        
        observation = self._get_observation()
        reward, terminated = self._calculate_reward()
        
        pos = self._get_current_position()
        target = self._get_target()
        distance = np.linalg.norm(pos - target)
        self.episode_distances.append(distance)
        self.episode_min_distance = min(self.episode_distances)
        truncated = False
        if self.current_step >= self.current_params['max_steps']:
            truncated = True
        #if truncated:
        #    reward += 10
        """
        if self.training_stage == 0:
            # If we ended the episode by hitting max_steps (truncate) WITHOUT crashing (terminated=False),
            # then we survived. Increment the counter.
            if truncated:
                self.hover_survival_count += 1
                reward += 50

                # If we've survived enough times in a row, move to stage 1
                if self.hover_survival_count >= self.num_consecutive_survivals_needed:
                    self.training_stage = 1
                    self.setup_stage_params()
                    self.hover_survival_count = 0
                    print(
                        f"=== Moved to stage {self.training_stage} after "
                        f"{self.num_consecutive_survivals_needed} consecutive survival episodes. ==="
                    )
        """
        return observation, reward, terminated, truncated, {"min_distance": self.episode_min_distance}
    def _calculate_reward(self):
        pos = self._get_current_position()
        vel = self._get_current_velocity()
        angular_vel = self._get_imu_data()[:3]
        target = self._get_target()
        distance = np.linalg.norm(pos - target)
        reward_scale = 0.005
        reward = 0.0
        # Immediate failure conditions (keep these)
        if self.client.simGetCollisionInfo().has_collided:
            reward -= 200*reward_scale
            return reward, True
        if any(abs(p) > a for p, a in zip(pos, self.current_params['area'])):
            reward -= 100*reward_scale
            return reward, True
        # Main distance reward - smoother gradient
        distance_reward = -distance  # Linear scaling for more consistent gradient
        reward += distance_reward * reward_scale
        
        """
        # Stability penalty - much gentler
        stability_penalty = (
            0.05 * np.linalg.norm(angular_vel) +  # Reduced rotation penalty
            0.02 * np.linalg.norm(vel)            # Reduced velocity penalty
        )
        reward -= stability_penalty
        
        if self.training_stage == 0:  # Hovering
            # Pure hovering - focus on stability
            vertical_distance = abs(pos[2] - target[2])
            reward += (-vertical_distance + 1.0)*reward_scale
            reward += (distance_reward*0.5)*reward_scale
            
            
        elif self.training_stage == 1:  # Vertical movement
            # Reward vertical progress toward target
            vertical_distance = abs(pos[2] - target[2])
            reward += (-vertical_distance + 1.0)*reward_scale
            reward += (distance_reward*0.5)*reward_scale
            # Penalize horizontal drift more strongly
            horizontal_drift = np.linalg.norm(pos[:2] - target[:2])
            reward -= (0.5 * horizontal_drift)*reward_scale
            
            reward += (distance_reward*0.5)*reward_scale
            
        elif self.training_stage == 2:  # Close-range horizontal
            # Reward horizontal progress
            if hasattr(self, '_prev_distance'):
                progress = self._prev_distance - distance
                reward += (3.0 * progress)*reward_scale  # Stronger reward for deliberate movement
            self._prev_distance = distance
            
            # Height stability becomes secondary but still important
            reward += (distance_reward)*reward_scale
            
        elif self.training_stage >= 3:  # Medium-range movement
            # Balance distance and control
            if hasattr(self, '_prev_distance'):
                progress = self._prev_distance - distance
                reward += (2.0 * progress)*reward_scale
            self._prev_distance = distance
            reward += (distance_reward)*reward_scale
            
        # Success condition
        success_radius = self.current_params['target_radius']
        if distance < success_radius:
            reward += (10.0)*reward_scale
            self.consecutive_success += 1
            if self.consecutive_success >= self.success_threshold:
                self.current_target = self._generate_new_target()
                reward += 200.0*reward_scale  
                self.success_count += 1
                return reward, True
            if self.success_count >= self.stage_threshold:
                self.training_stage = min(8, self.training_stage + 1)
                print(f"\n=== Advanced to stage {self.training_stage} ===")
                self.success_count = 0
                self.setup_stage_params()
                return reward, True
        return reward, False
        """
        if self.success_count >= self.stage_threshold:
            self.training_stage = min(9, self.training_stage + 1)
            print(f"\n=== Advanced to stage {self.training_stage} ===")
            self.success_count = 0
            self.setup_stage_params()
            return reward, True
        success_radius = self.current_params['target_radius']
        if distance <= success_radius:
            reward += (2_000.0)*reward_scale
            self.success_count += 1
            return reward, True
        return reward, False
        
        
    def close(self):
        self.client.reset()
        self.client.enableApiControl(False)
        self.client.armDisarm(False)

In [ ]:
# Print device info
print(f"Using device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

# Create log directory
log_dir = f"./drone_training_logs/{datetime.now().strftime('%Y%m%d_%H%M%S')}"
os.makedirs(log_dir, exist_ok=True)

# Environment setup
def make_env():
    """Helper function to create and wrap the AirSim environment"""
    env = CosysAirSimEnv_Velocity(training_stage=0)  # Start with hover training
    env = Monitor(env)
    return env

# Create and wrap the environment
env = DummyVecEnv([make_env])
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.0, clip_reward=10.0)

# Create evaluation environment
eval_env = DummyVecEnv([make_env])
eval_env = VecNormalize(eval_env, norm_obs=True, norm_reward=True, clip_obs=10.0, clip_reward=10.0)

# Create evaluation callback
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=f"{log_dir}/best_model",
    log_path=f"{log_dir}/eval_results",
    eval_freq=5000,
    n_eval_episodes=5,
    deterministic=True,
    render=False
)

# Initialize the PPO model
model = PPO(
    "MlpPolicy",
    env,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=64,
    n_epochs=10,
    gamma=0.99,
    gae_lambda=0.95,
    clip_range=0.2,
    ent_coef=0.01,
    policy_kwargs=dict(
        net_arch=dict(
            pi=[256, 256],
            vf=[256, 256]
        ),
        activation_fn=torch.nn.ReLU
    ),
    verbose=1,
    tensorboard_log=f"{log_dir}/tensorboard"
)

# Custom callback for plotting
class NotebookProgressCallback(BaseCallback):
    def __init__(self, check_freq=1000, verbose=1):
        super().__init__(verbose)
        self.check_freq = check_freq
        self.rewards = []
        self.episodes = []
        self.steps = []
        
    def _on_step(self) -> bool:
        if len(self.training_env.buf_rews) > 0:
            episode_reward = self.training_env.buf_rews[0]
            self.rewards.append(episode_reward)
            self.episodes.append(len(self.rewards))
            self.steps.append(self.num_timesteps)
            
            if len(self.rewards) % self.check_freq == 0:
                clear_output(wait=True)
                plt.figure(figsize=(10, 5))
                plt.plot(self.episodes, self.rewards)
                plt.title('Training Progress')
                plt.xlabel('Episode')
                plt.ylabel('Reward')
                plt.show()
                
                print(f"Steps: {self.num_timesteps}")
                print(f"Mean reward over last 100 episodes: {np.mean(self.rewards[-100:]):.2f}")
                print(f"Current training stage: {self.training_env.envs[0].env.training_stage}")
        
        return True

# Training function
def train_drone(total_timesteps=2_000_000):
    progress_callback = NotebookProgressCallback()
    try:
        model.learn(
            total_timesteps=total_timesteps,
            callback=[eval_callback, progress_callback],
            progress_bar=True
        )
        
        # Save final model
        model.save(f"{log_dir}/final_model")
        env.save(f"{log_dir}/vec_normalize.pkl")
        print("\nTraining completed successfully!")
        
    except KeyboardInterrupt:
        print("\nTraining interrupted! Saving current model...")
        model.save(f"{log_dir}/interrupted_model")
        env.save(f"{log_dir}/vec_normalize.pkl")
        print("Model saved!")
    
    finally:
        env.close()
        eval_env.close()

# Evaluation function
def evaluate_model(model_path, vec_normalize_path, n_eval_episodes=10):
    """Function to evaluate a trained model"""
    # Load the saved statistics
    eval_env = DummyVecEnv([make_env])
    eval_env = VecNormalize.load(vec_normalize_path, eval_env)
    eval_env.training = False
    eval_env.norm_reward = False
    
    # Load the model
    loaded_model = PPO.load(model_path)
    
    rewards = []
    for episode in range(n_eval_episodes):
        obs = eval_env.reset()
        done = False
        total_reward = 0
        
        while not done:
            action, _ = loaded_model.predict(obs, deterministic=True)
            obs, reward, done, info = eval_env.step(action)
            total_reward += reward
            
        rewards.append(total_reward)
        print(f"Episode {episode + 1}: Reward = {total_reward}")
    
    print(f"\nAverage reward over {n_eval_episodes} episodes: {np.mean(rewards):.2f}")
    
    # Plot evaluation results
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, n_eval_episodes + 1), rewards, 'bo-')
    plt.title('Evaluation Results')
    plt.xlabel('Episode')
    plt.ylabel('Reward')
    plt.grid(True)
    plt.show()
    
    eval_env.close()

In [ ]:
train_drone(total_timesteps=2_000_000)

In [57]:
class ReplayBuffer:
    def __init__(self, max_size, obs_dim, act_dim, device="cuda" if torch.cuda.is_available() else "cpu"):
        self.max_size = max_size
        self.ptr = 0
        self.size = 0
        self.device = device
        # Use float32 for all arrays to match network precision
        self.states = np.zeros((max_size, obs_dim), dtype=np.float32)
        self.actions = np.zeros((max_size, act_dim), dtype=np.float32)
        self.rewards = np.zeros(max_size, dtype=np.float32)  # Flattened array
        self.next_states = np.zeros((max_size, obs_dim), dtype=np.float32)
        self.dones = np.zeros(max_size, dtype=np.float32)    # Flattened array

    def add(self, state, action, reward, next_state, done):
        # Ensure incoming data is on CPU and in numpy format
        if torch.is_tensor(state):
            state = state.detach().cpu().numpy()
        if torch.is_tensor(action):
            action = action.detach().cpu().numpy()
        if torch.is_tensor(reward):
            reward = reward.detach().cpu().numpy()
        if torch.is_tensor(next_state):
            next_state = next_state.detach().cpu().numpy()
        if torch.is_tensor(done):
            done = done.detach().cpu().numpy()
            
        np.copyto(self.states[self.ptr], state)
        np.copyto(self.actions[self.ptr], action)
        self.rewards[self.ptr] = reward
        np.copyto(self.next_states[self.ptr], next_state)
        self.dones[self.ptr] = done
        
        self.ptr = (self.ptr + 1) % self.max_size
        self.size = min(self.size + 1, self.max_size)
    
    def sample(self, batch_size):
        ind = np.random.randint(0, self.size, size=batch_size)
        
        return (
            torch.FloatTensor(self.states[ind]).to(self.device),
            torch.FloatTensor(self.actions[ind]).to(self.device),
            torch.FloatTensor(self.rewards[ind]).to(self.device),
            torch.FloatTensor(self.next_states[ind]).to(self.device),
            torch.FloatTensor(self.dones[ind]).to(self.device)
        )
    
    def save(self, path):
        path = Path(path)
        path.parent.mkdir(parents=True, exist_ok=True)
        
        # Save the buffer state and metadata
        save_dict = {
            'max_size': self.max_size,
            'ptr': self.ptr,
            'size': self.size,
            'states': self.states,
            'actions': self.actions,
            'rewards': self.rewards,
            'next_states': self.next_states,
            'dones': self.dones,
            'device': self.device
        }
        
        try:
            with open(path, 'wb') as f:
                pickle.dump(save_dict, f)
        except Exception as e:
            raise RuntimeError(f"Failed to save buffer to {path}: {e}")

    @staticmethod
    def load(path, device=None):
        try:
            with open(path, 'rb') as f:
                save_dict = pickle.load(f)
            
            # Create new buffer with saved dimensions
            obs_dim = save_dict['states'].shape[1]
            act_dim = save_dict['actions'].shape[1]
            buffer = ReplayBuffer(
                max_size=save_dict['max_size'],
                obs_dim=obs_dim,
                act_dim=act_dim,
                device=device or save_dict['device']
            )
            
            # Restore buffer state
            buffer.ptr = save_dict['ptr']
            buffer.size = save_dict['size']
            buffer.states = save_dict['states']
            buffer.actions = save_dict['actions']
            buffer.rewards = save_dict['rewards']
            buffer.next_states = save_dict['next_states']
            buffer.dones = save_dict['dones']
            
            return buffer
        except Exception as e:
            raise RuntimeError(f"Failed to load buffer from {path}: {e}")
        
    def __len__(self):
        return self.size
"""
class ReplayBuffer:
    def __init__(self, max_size, obs_dim, act_dim, device="cuda" if torch.cuda.is_available() else "cpu"):
        self.max_size = max_size
        self.ptr = 0
        self.size = 0
        self.device = device

        # Initialize buffers with correct shapes
        self.states = np.zeros((max_size, obs_dim), dtype=np.float32)
        self.actions = np.zeros((max_size, act_dim), dtype=np.float32)
        self.rewards = np.zeros((max_size, 1), dtype=np.float32)  # Changed shape to (max_size, 1)
        self.next_states = np.zeros((max_size, obs_dim), dtype=np.float32)
        self.dones = np.zeros((max_size, 1), dtype=np.float32)    # Changed shape to (max_size, 1)

    def add(self, state, action, reward, next_state, done):
        # Convert inputs to numpy arrays and ensure correct shapes
        state = np.array(state, dtype=np.float32).flatten()
        action = np.array(action, dtype=np.float32).flatten()
        reward = np.array(reward, dtype=np.float32).reshape(1)
        next_state = np.array(next_state, dtype=np.float32).flatten()
        done = np.array(done, dtype=np.float32).reshape(1)

        # Store transition
        self.states[self.ptr] = state
        self.actions[self.ptr] = action
        self.rewards[self.ptr] = reward
        self.next_states[self.ptr] = next_state
        self.dones[self.ptr] = done

        # Update pointer and size
        self.ptr = (self.ptr + 1) % self.max_size
        self.size = min(self.size + 1, self.max_size)

    def sample(self, batch_size):
        ind = np.random.randint(0, self.size, size=batch_size)
        
        return (
            torch.FloatTensor(self.states[ind]).to(self.device),
            torch.FloatTensor(self.actions[ind]).to(self.device),
            torch.FloatTensor(self.rewards[ind]).to(self.device),
            torch.FloatTensor(self.next_states[ind]).to(self.device),
            torch.FloatTensor(self.dones[ind]).to(self.device)
        )
    def save(self, path):
        path = Path(path)
        path.parent.mkdir(parents=True, exist_ok=True)
        
        # Save the buffer state and metadata
        save_dict = {
            'max_size': self.max_size,
            'ptr': self.ptr,
            'size': self.size,
            'states': self.states,
            'actions': self.actions,
            'rewards': self.rewards,
            'next_states': self.next_states,
            'dones': self.dones,
            'device': self.device
        }
        
        try:
            with open(path, 'wb') as f:
                pickle.dump(save_dict, f)
        except Exception as e:
            raise RuntimeError(f"Failed to save buffer to {path}: {e}")

    @staticmethod
    def load(path, device=None):
        try:
            with open(path, 'rb') as f:
                save_dict = pickle.load(f)
            
            # Create new buffer with saved dimensions
            obs_dim = save_dict['states'].shape[1]
            act_dim = save_dict['actions'].shape[1]
            buffer = ReplayBuffer(
                max_size=save_dict['max_size'],
                obs_dim=obs_dim,
                act_dim=act_dim,
                device=device or save_dict['device']
            )
            
            # Restore buffer state
            buffer.ptr = save_dict['ptr']
            buffer.size = save_dict['size']
            buffer.states = save_dict['states']
            buffer.actions = save_dict['actions']
            buffer.rewards = save_dict['rewards']
            buffer.next_states = save_dict['next_states']
            buffer.dones = save_dict['dones']
            
            return buffer
        except Exception as e:
            raise RuntimeError(f"Failed to load buffer from {path}: {e}")
        
    def __len__(self):
        return self.size
"""
class TD3Actor(nn.Module):
    def __init__(self, obs_dim, act_dim, max_action, hidden_dim=1024): #512
        super().__init__()
        """
        self.feature_net = nn.Sequential(
            nn.Linear(obs_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim), #was LayerNorm
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim), 
            nn.BatchNorm1d(hidden_dim), #was LayerNorm
            nn.ReLU()
        )
        
        self.action_net = nn.Sequential(
            nn.Linear(hidden_dim, act_dim),
            nn.Tanh()
        )
        
        self.max_action = max_action
        self.__init_args__ = (obs_dim, act_dim, max_action, hidden_dim)
        self._init_weights()
        """

        self.net = nn.Sequential(
            nn.Linear(obs_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, act_dim),
            nn.Tanh()
        )
        
        self.max_action = max_action
        self.__init_args__ = (obs_dim, act_dim, max_action, hidden_dim)
        
        
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.orthogonal_(m.weight, gain=np.sqrt(2))
                nn.init.constant_(m.bias, 0.0)

    def forward(self, state):
        return self.max_action * self.net(state)

class TD3Critic(nn.Module):
    def __init__(self, obs_dim, act_dim, hidden_dim=1024): #512
        super().__init__()
        """
        self.state_net = nn.Sequential(
            nn.Linear(obs_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim), #was LayerNorm
            nn.ReLU(),
        )
        
        self.action_net = nn.Sequential(
            nn.Linear(act_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim), #was LayerNorm
            nn.ReLU(),
        )
        
        self.q_net = nn.Sequential(
            nn.Linear(2 * hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
        
        self.__init_args__ = (obs_dim, act_dim, hidden_dim)
        self._init_weights()
        """
        self.net = nn.Sequential(
            nn.Linear(obs_dim + act_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
        
        self.__init_args__ = (obs_dim, act_dim, hidden_dim)
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.orthogonal_(m.weight, gain=1.0)
                nn.init.constant_(m.bias, 0)

    def forward(self, state, action):
        sa = torch.cat([state, action], dim=1)
        q = self.net(sa)
        return q
    """
class TD3Actor(nn.Module):
    def __init__(self, obs_dim, act_dim, max_action, hidden_dim=256):
        super().__init__()
        
        self.net = nn.Sequential(
            nn.Linear(obs_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, act_dim),
            nn.Tanh()
        )
        
        self.max_action = max_action
        self.__init_args__ = (obs_dim, act_dim, max_action, hidden_dim)
        
        # Use orthogonal initialization
        for layer in self.net:
            if isinstance(layer, nn.Linear):
                nn.init.orthogonal_(layer.weight, gain=np.sqrt(2))
                nn.init.constant_(layer.bias, 0.0)

    def forward(self, state):
        return self.max_action * self.net(state)

class TD3Critic(nn.Module):
    def __init__(self, obs_dim, act_dim, hidden_dim=256):
        super().__init__()
        
        # Q1 architecture
        self.q1_net = nn.Sequential(
            nn.Linear(obs_dim + act_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
        
        # Q2 architecture
        self.q2_net = nn.Sequential(
            nn.Linear(obs_dim + act_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
        
        self.__init_args__ = (obs_dim, act_dim, hidden_dim)
        self._init_weights()

    def _init_weights(self):
        for net in [self.q1_net, self.q2_net]:
            for m in net.modules():
                if isinstance(m, nn.Linear):
                    nn.init.orthogonal_(m.weight, gain=np.sqrt(2))
                    nn.init.constant_(m.bias, 0)

    def forward(self, state, action):
        # Ensure inputs are properly shaped
        if state.dim() == 1:
            state = state.unsqueeze(0)
        if action.dim() == 1:
            action = action.unsqueeze(0)
            
        # Concatenate state and action
        sa = torch.cat([state, action], dim=1)
        
        # Get Q-values
        q1 = self.q1_net(sa)
        q2 = self.q2_net(sa)
        
        return q1, q2

    def Q1(self, state, action):
        # Ensure inputs are properly shaped
        if state.dim() == 1:
            state = state.unsqueeze(0)
        if action.dim() == 1:
            action = action.unsqueeze(0)
            
        # Concatenate state and action
        sa = torch.cat([state, action], dim=1)
        
        return self.q1_net(sa)
"""
class TD3Trainer:
    def __init__(self, actor, critic1, critic2, actor_optimizer, critic_optimizer1, 
                 critic_optimizer2, max_action, device, gamma=0.99, tau=0.001):
        
        self.actor = actor
        self.critic1 = critic1
        self.critic2 = critic2
        self.actor_target = type(actor)(*actor.__init_args__).to(device)
        self.critic_target1 = type(critic1)(*critic1.__init_args__).to(device)
        self.critic_target2 = type(critic2)(*critic2.__init_args__).to(device)
        
        self.actor_optimizer = actor_optimizer
        self.critic_optimizer1 = critic_optimizer1
        self.critic_optimizer2 = critic_optimizer2
        
        self.max_action = max_action
        self.device = device
        self.gamma = gamma
        self.tau = tau
        
        self._hard_update_targets()
        
        self.target_entropy = -float(actor.__init_args__[1])
        
        # Initialize episode tracking
        self.current_episode_rewards = []
        self.episode_returns = []
        
    def _hard_update_targets(self):
        self.actor_target.load_state_dict(self.actor.state_dict())
        self.critic_target1.load_state_dict(self.critic1.state_dict())
        self.critic_target2.load_state_dict(self.critic2.state_dict())
    
    def _soft_update(self, target, source):
        with torch.no_grad():
            for target_param, param in zip(target.parameters(), source.parameters()):
                target_param.data.copy_(
                    self.tau * param.data + (1.0 - self.tau) * target_param.data
                )

    def train_step(self, replay_buffer, batch_size, noise_std, noise_clip, policy_delay, total_it):
        state, action, reward, next_state, done = replay_buffer.sample(batch_size)
        reward = reward.view(-1, 1)
        done = done.view(-1, 1)

        with torch.no_grad():
            noise = (torch.randn_like(action) * noise_std).clamp(-noise_clip, noise_clip)
            next_action = (self.actor_target(next_state) + noise).clamp(-self.max_action, self.max_action)
            
            target_Q1 = self.critic_target1(next_state, next_action)
            target_Q2 = self.critic_target2(next_state, next_action)
            target_Q = torch.min(target_Q1, target_Q2)
            target_Q = reward + (1 - done) * self.gamma * target_Q
        
        # Critic 1 update
        current_Q1 = self.critic1(state, action)
        critic_mse_loss1 = F.mse_loss(current_Q1, target_Q)
        critic_l2_reg1 = 0 #0.00001 * sum(torch.sum(param ** 2) for param in self.critic1.parameters())
        critic_loss1 = critic_mse_loss1 + critic_l2_reg1
        
        self.critic_optimizer1.zero_grad()
        critic_loss1.backward() 
        torch.nn.utils.clip_grad_norm_(self.critic1.parameters(), 1.0)
        self.critic_optimizer1.step()
        
        # Critic 2 update
        current_Q2 = self.critic2(state, action)
        critic_mse_loss2 = F.mse_loss(current_Q2, target_Q)
        critic_l2_reg2 = 0 #0.00001 * sum(torch.sum(param ** 2) for param in self.critic2.parameters())
        critic_loss2 = critic_mse_loss2 + critic_l2_reg2
        
        self.critic_optimizer2.zero_grad()
        critic_loss2.backward()  
        torch.nn.utils.clip_grad_norm_(self.critic2.parameters(), 1.0)
        self.critic_optimizer2.step()

        actor_loss = None
        if total_it % policy_delay == 0:
            actor_actions = self.actor(state)
            actor_loss = -self.critic1(state, actor_actions).mean()
            
            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            torch.nn.utils.clip_grad_norm_(self.actor.parameters(), 1.0)
            self.actor_optimizer.step()
            
            self._soft_update(self.actor_target, self.actor)
            self._soft_update(self.critic_target1, self.critic1)
            self._soft_update(self.critic_target2, self.critic2)
        
        return {
            'actor_loss': actor_loss.item() if actor_loss is not None else None,
            'critic_loss1': critic_loss1.item(),
            'critic_loss2': critic_loss2.item(),
            'q_values1': target_Q1,
            'q_values2': target_Q2,
            'target_q_values': target_Q
        }
    """
class TD3Trainer:
    def __init__(self, actor, critic1, critic2, actor_optimizer, critic_optimizer1, 
                 critic_optimizer2, max_action, device, gamma=0.99, tau=0.005):
        
        self.actor = actor
        self.critic1 = critic1
        self.critic2 = critic2
        self.actor_target = type(actor)(*actor.__init_args__).to(device)
        self.critic_target1 = type(critic1)(*critic1.__init_args__).to(device)
        self.critic_target2 = type(critic2)(*critic2.__init_args__).to(device)
        
        self.actor_optimizer = actor_optimizer
        self.critic_optimizer1 = critic_optimizer1
        self.critic_optimizer2 = critic_optimizer2
        
        self.max_action = max_action
        self.device = device
        self.gamma = gamma
        self.tau = tau
        
        self._hard_update_targets()

    def _hard_update_targets(self):
        self.actor_target.load_state_dict(self.actor.state_dict())
        self.critic_target1.load_state_dict(self.critic1.state_dict())
        self.critic_target2.load_state_dict(self.critic2.state_dict())
    
    def _soft_update(self, target, source):
        with torch.no_grad():
            for target_param, param in zip(target.parameters(), source.parameters()):
                target_param.data.copy_(
                    self.tau * param.data + (1.0 - self.tau) * target_param.data
                )

    def train_step(self, replay_buffer, batch_size, noise_std, noise_clip, policy_delay, total_it):
        # Sample from replay buffer
        state, action, reward, next_state, done = replay_buffer.sample(batch_size)
        
        # Ensure proper dimensions
        reward = reward.view(-1, 1)
        done = done.view(-1, 1)

        with torch.no_grad():
            # Select action according to policy and add clipped noise
            noise = (torch.randn_like(action) * noise_std).clamp(-noise_clip, noise_clip)
            next_action = (self.actor_target(next_state) + noise).clamp(-self.max_action, self.max_action)
            
            # Compute the target Q value
            target_Q1, target_Q2 = self.critic_target1(next_state, next_action)
            target_Q = torch.min(target_Q1, target_Q2)
            target_Q = reward + (1 - done) * self.gamma * target_Q

        # Get current Q estimates
        current_Q1, _ = self.critic1(state, action)
        current_Q2, _ = self.critic2(state, action)

        # Compute critic loss
        critic_loss1 = F.mse_loss(current_Q1, target_Q)
        critic_loss2 = F.mse_loss(current_Q2, target_Q)

        # Optimize the critics
        self.critic_optimizer1.zero_grad()
        critic_loss1.backward()
        self.critic_optimizer1.step()

        self.critic_optimizer2.zero_grad()
        critic_loss2.backward()
        self.critic_optimizer2.step()

        actor_loss = None

        # Delayed policy updates
        if total_it % policy_delay == 0:
            # Compute actor loss
            actor_action = self.actor(state)
            Q1, _ = self.critic1(state, actor_action)
            actor_loss = -Q1.mean()

            # Optimize the actor
            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            self.actor_optimizer.step()

            # Update the frozen target models
            self._soft_update(self.actor_target, self.actor)
            self._soft_update(self.critic_target1, self.critic1)
            self._soft_update(self.critic_target2, self.critic2)

        return {
            'critic_loss1': critic_loss1.item(),
            'critic_loss2': critic_loss2.item(),
            'actor_loss': actor_loss.item() if actor_loss is not None else None
        }
    """

In [63]:
def compute_gradient_norm(model):
    total_norm = 0
    for p in model.parameters():
        if p.grad is not None:
            param_norm = p.grad.data.norm(2)
            total_norm += param_norm.item() ** 2
    return total_norm ** 0.5

def compute_parameter_norm(model):
    total_norm = 0
    for p in model.parameters():
        param_norm = p.data.norm(2)
        total_norm += param_norm.item() ** 2
    return total_norm ** 0.5

def evaluate_policy(actor, env, num_episodes=25, device='cuda'):
    """Evaluate the policy without exploration noise"""
    eval_rewards = []
    eval_success = 0
    
    for _ in range(num_episodes):
        state, _ = env.reset()
        episode_reward = 0
        done = False
        
        while not done:
            with torch.no_grad():
                state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
                action = actor(state_tensor).cpu().numpy().flatten()
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            episode_reward += reward
            state = next_state
            
            if info.get('success', False):
                eval_success += 1
                break
                
        eval_rewards.append(episode_reward)
    
    return {
        'mean_reward': np.mean(eval_rewards),
        'std_reward': np.std(eval_rewards),
        'success_rate': eval_success / num_episodes,
        'rewards': eval_rewards
    }

def run_training(env, obs_dim, act_dim, max_action, episodes=10000):
    # Initialize environment and models
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Initialize TensorBoard writer with more descriptive name
    current_time = datetime.now().strftime('%Y%m%d-%H%M%S')
    log_dir = os.path.join('logs', f'TD3_train_{current_time}')
    train_writer = SummaryWriter(log_dir + '/train')
    eval_writer = SummaryWriter(log_dir + '/eval')
    print(f"TensorBoard logs will be saved to: {log_dir}")
    
    # Model initialization
    actor = TD3Actor(obs_dim, act_dim, max_action).to(device)
    critic1 = TD3Critic(obs_dim, act_dim).to(device)
    critic2 = TD3Critic(obs_dim, act_dim).to(device)
    
    actor_optimizer = optim.Adam(actor.parameters(), lr=3e-6) #3e-4
    critic_optimizer1 = optim.Adam(critic1.parameters(), lr=3e-6)
    critic_optimizer2 = optim.Adam(critic2.parameters(), lr=3e-6)

    # Load checkpoint if exists
    total_steps = 0
    current_stage = 0
    best_rewards = {i: float('-inf') for i in range(9)}
    
    try:
        checkpoint = torch.load('models/TD3/Model/mtp_backup_2250.pth')
        actor.load_state_dict(checkpoint['actor_state_dict'])
        critic1.load_state_dict(checkpoint['critic1_state_dict'])
        critic2.load_state_dict(checkpoint['critic2_state_dict'])
        actor_optimizer.load_state_dict(checkpoint['actor_optimizer_state_dict'])
        critic_optimizer1.load_state_dict(checkpoint['critic1_optimizer_state_dict'])
        critic_optimizer2.load_state_dict(checkpoint['critic2_optimizer_state_dict'])
        
        try:
            total_steps = checkpoint['total_steps']
            current_stage = checkpoint['stage']
            best_rewards = checkpoint['best_rewards']
        except KeyError:
            pass
        print(f"Loaded checkpoint successfully. Total steps: {total_steps}")
    except:
        print("No checkpoint found, starting fresh")

    # Initialize or load replay buffer
    try:
        replay_buffer = ReplayBuffer.load('models/TD3/Replay_Buffer/mtp_replay_buffer.pkl')
        print("Loaded buffer successfully")
    except:
        replay_buffer = ReplayBuffer(max_size=1_000_000, obs_dim=obs_dim, act_dim=act_dim)
        print("No buffer found, starting fresh")
    
    trainer = TD3Trainer(
        actor=actor,
        critic1=critic1,
        critic2=critic2,
        actor_optimizer=actor_optimizer,
        critic_optimizer1=critic_optimizer1,
        critic_optimizer2=critic_optimizer2,
        max_action=max_action,
        device=device
    )
    trainer._hard_update_targets()

    # Training hyperparameters
    batch_size = 512 #256
    warmup_steps = 0 #20_000 #40_000
    noise_std = 0.1
    noise_clip = 0.75
    policy_delay = 2
    eval_freq = 200  # Evaluate every 200 episodes
    
    # Create directories
    os.makedirs('models/TD3/Replay_Buffer', exist_ok=True)
    os.makedirs('models/TD3/Model', exist_ok=True)
    os.makedirs('logs', exist_ok=True)
    
    # Training metrics tracking
    episode_rewards = []
    recent_rewards = deque(maxlen=100)
    stage_rewards = {i: [] for i in range(9)}
    best_eval_reward = float('-inf')
    
    def get_exploration_noise(action):
        action_tensor = torch.from_numpy(action)
        base_noise = (torch.randn_like(action_tensor) * noise_std).clamp(-noise_clip, noise_clip)
        return base_noise.cpu().numpy()
    
    def log_training_step(train_info, step):
        """
        Enhanced logging function that tracks detailed metrics during training
        """
        # Loss logging
        train_writer.add_scalar('Loss/Critic1', train_info['critic_loss1'], step)
        train_writer.add_scalar('Loss/Critic2', train_info['critic_loss2'], step)
        if train_info['actor_loss'] is not None:
            train_writer.add_scalar('Loss/Actor', train_info['actor_loss'], step)
        
        # Detailed gradient norm logging for each network
        for name, param in actor.named_parameters():
            if param.grad is not None:
                train_writer.add_scalar(f'Gradients/Actor/{name}_grad_norm', 
                                    param.grad.norm(2).item(), step)
        
        for name, param in critic1.named_parameters():
            if param.grad is not None:
                train_writer.add_scalar(f'Gradients/Critic1/{name}_grad_norm', 
                                    param.grad.norm(2).item(), step)
        
        for name, param in critic2.named_parameters():
            if param.grad is not None:
                train_writer.add_scalar(f'Gradients/Critic2/{name}_grad_norm', 
                                    param.grad.norm(2).item(), step)
        
        # Overall gradient norms
        train_writer.add_scalar('Gradients/Actor_Total_Norm', compute_gradient_norm(actor), step)
        train_writer.add_scalar('Gradients/Critic1_Total_Norm', compute_gradient_norm(critic1), step)
        train_writer.add_scalar('Gradients/Critic2_Total_Norm', compute_gradient_norm(critic2), step)
        
        # Parameter value norms for each layer
        for name, param in actor.named_parameters():
            train_writer.add_scalar(f'Parameters/Actor/{name}_norm', 
                                param.norm(2).item(), step)
        
        for name, param in critic1.named_parameters():
            train_writer.add_scalar(f'Parameters/Critic1/{name}_norm', 
                                param.norm(2).item(), step)
        
        for name, param in critic2.named_parameters():
            train_writer.add_scalar(f'Parameters/Critic2/{name}_norm', 
                                param.norm(2).item(), step)
        
        # Overall parameter norms
        train_writer.add_scalar('Parameters/Actor_Total_Norm', compute_parameter_norm(actor), step)
        train_writer.add_scalar('Parameters/Critic1_Total_Norm', compute_parameter_norm(critic1), step)
        train_writer.add_scalar('Parameters/Critic2_Total_Norm', compute_parameter_norm(critic2), step)
        
        # Learning rates
        train_writer.add_scalar('LearningRate/Actor', actor_optimizer.param_groups[0]['lr'], step)
        train_writer.add_scalar('LearningRate/Critic1', critic_optimizer1.param_groups[0]['lr'], step)
        train_writer.add_scalar('LearningRate/Critic2', critic_optimizer2.param_groups[0]['lr'], step)
        
        # Q-value statistics from critics
        if 'q_values1' in train_info:
            train_writer.add_scalar('Q_Values/Critic1_Mean', train_info['q_values1'].mean().item(), step)
            train_writer.add_scalar('Q_Values/Critic1_Std', train_info['q_values1'].std().item(), step)
            train_writer.add_histogram('Q_Values/Critic1_Distribution', train_info['q_values1'], step)
        
        if 'q_values2' in train_info:
            train_writer.add_scalar('Q_Values/Critic2_Mean', train_info['q_values2'].mean().item(), step)
            train_writer.add_scalar('Q_Values/Critic2_Std', train_info['q_values2'].std().item(), step)
            train_writer.add_histogram('Q_Values/Critic2_Distribution', train_info['q_values2'], step)
        
        # Target Q-values if available
        if 'target_q_values' in train_info:
            train_writer.add_scalar('Q_Values/Target_Mean', train_info['target_q_values'].mean().item(), step)
            train_writer.add_scalar('Q_Values/Target_Std', train_info['target_q_values'].std().item(), step)
            train_writer.add_histogram('Q_Values/Target_Distribution', train_info['target_q_values'], step)
            
        if step % 100 == 0:
            train_writer.flush()
            eval_writer.flush()
    
    def run_evaluation(episode):
        # Create a separate environment for evaluation
        eval_env = env.__class__()  # Assuming env has a constructor that takes no arguments
        eval_env.training_stage = env.training_stage  # Sync the training stage
        
        # Run evaluation
        eval_results = evaluate_policy(actor, eval_env, num_episodes=10, device=device)
        
        # Log evaluation metrics
        eval_writer.add_scalar('Eval/Mean_Reward', eval_results['mean_reward'], episode)
        eval_writer.add_scalar('Eval/Reward_Std', eval_results['std_reward'], episode)
        eval_writer.add_scalar('Eval/Success_Rate', eval_results['success_rate'], episode)
        
        # Log reward distribution
        eval_writer.add_histogram('Eval/Reward_Distribution', 
                                torch.tensor(eval_results['rewards']), 
                                episode)
        
        eval_env.close()
        return eval_results
    
    progress_bar = tqdm.tqdm(range(episodes), desc="Training")
    for episode in progress_bar:
        state, _ = env.reset()
        episode_reward = 0
        done = False
        episode_steps = 0
        current_stage = env.training_stage
        episode_min_distance = float('inf')
        # Log current stage
        train_writer.add_scalar('Training/Current_Stage', current_stage, total_steps)
        
        while not done:
            # Select action
            if total_steps < warmup_steps:
                action = np.random.uniform(-max_action, max_action, size=act_dim)
                train_writer.add_scalar('Training/Exploration_Type', 0, total_steps)  # 0 for random
            else:
                with torch.no_grad():
                    state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
                    action = actor(state_tensor).cpu().numpy().flatten()
                    current_noise = get_exploration_noise(action)
                    action = action + current_noise
                action = np.clip(action, -max_action, max_action)
                train_writer.add_scalar('Training/Exploration_Type', 1, total_steps)  # 1 for policy
                train_writer.add_scalar('Training/Exploration_Noise_Mean', np.mean(current_noise), total_steps)
                train_writer.add_scalar('Training/Exploration_Noise_Std', np.std(current_noise), total_steps)
                train_writer.add_scalar('Training/Exploration_Noise_Max', np.max(np.abs(current_noise)), total_steps)
            
            # Log action statistics
            train_writer.add_histogram('Actions/Distribution', action, total_steps)
            train_writer.add_scalar('Actions/Mean', np.mean(action), total_steps)
            train_writer.add_scalar('Actions/Std', np.std(action), total_steps)
            
            # Step environment
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            
            if "min_distance" in info:
                episode_min_distance = min(episode_min_distance, info["min_distance"])
            
            # Store transition
            replay_buffer.add(state, action, reward, next_state, float(done))
            
            # Update state and metrics
            state = next_state
            episode_reward += reward
            total_steps += 1
            episode_steps += 1
            
            # Train agent
            if total_steps > warmup_steps and len(replay_buffer.states) > batch_size:
                train_info = trainer.train_step(
                    replay_buffer=replay_buffer,
                    batch_size=batch_size,
                    noise_std=noise_std,
                    noise_clip=noise_clip,
                    policy_delay=policy_delay,
                    total_it=total_steps,
                )
                
                # Log training metrics
                log_training_step(train_info, total_steps)
        
        # Episode completion logging
        recent_rewards.append(episode_reward)
        episode_rewards.append(episode_reward)
        stage_rewards[current_stage].append(episode_reward)
        
        # Log episode metrics
        train_writer.add_scalar('Episode/Reward', episode_reward, episode)
        train_writer.add_scalar('Episode/Steps', episode_steps, episode)
        train_writer.add_scalar('Episode/Average_100_Episodes', np.mean(recent_rewards), episode)
        train_writer.add_scalar('Training/Success_Count', env.success_count, episode)
        train_writer.add_scalar('Training/Buffer_Size', len(replay_buffer), episode)
        train_writer.add_scalar('Episode/Min_Distance', episode_min_distance, episode)
        # Run evaluation periodically
        """
        if episode % eval_freq == 0 and episode >= 800:
            eval_results = run_evaluation(episode)
            
            # Save best model based on evaluation
            if eval_results['mean_reward'] > best_eval_reward:
                best_eval_reward = eval_results['mean_reward']
                torch.save({
                    'actor_state_dict': actor.state_dict(),
                    'critic1_state_dict': critic1.state_dict(),
                    'critic2_state_dict': critic2.state_dict(),
                    'actor_optimizer_state_dict': actor_optimizer.state_dict(),
                    'critic1_optimizer_state_dict': critic_optimizer1.state_dict(),
                    'critic2_optimizer_state_dict': critic_optimizer2.state_dict(),
                    'total_steps': total_steps,
                    'episode': episode,
                    'stage': current_stage,
                    'eval_reward': best_eval_reward,
                    'best_rewards': best_rewards
                }, 'models/TD3/Model/mtp_model_best_eval.pth')
        """
        # Stage-specific metrics
        if len(stage_rewards[current_stage]) >= 100:
            current_avg_reward = np.mean(stage_rewards[current_stage][-100:])
            train_writer.add_scalar(f'Stage_{current_stage}/Average_100_Episodes', 
                                  current_avg_reward, 
                                  len(stage_rewards[current_stage]))
            
            if current_avg_reward > best_rewards[current_stage]:
                best_rewards[current_stage] = current_avg_reward
                train_writer.add_scalar(f'Stage_{current_stage}/Best_Average_Reward', 
                                      current_avg_reward, 
                                      len(stage_rewards[current_stage]))
        
        # Update progress bar
        progress_bar.set_postfix({
            'stage': current_stage,
            'reward': f'{episode_reward:.2f}',
            'avg_reward': f'{np.mean(recent_rewards):.2f}' if episode_rewards else 'N/A',
            'success_count': env.success_count,
            'min_dist': f'{episode_min_distance:.2f}' if episode_min_distance != float("inf") else 'N/A'
        })
        
        # Check for training completion
        if current_stage == 9 and env.success_count >= env.stage_threshold:
            print("\n=== Training completed successfully! ===")
            env.close()
            print("Environment closed. Running final evaluation...")
            final_eval_results = run_evaluation(episode)
            print(f"Final evaluation results: {final_eval_results}")
            train_writer.close()
            eval_writer.close()
            
            replay_buffer.save('models/TD3/Replay_Buffer/mtp_replay_buffer_final.pkl')
            checkpoint_path = f'models/TD3/Model/mtp_model_final.pth'
            torch.save({
                    'actor_state_dict': actor.state_dict(),
                    'critic1_state_dict': critic1.state_dict(),
                    'critic2_state_dict': critic2.state_dict(),
                    'actor_optimizer_state_dict': actor_optimizer.state_dict(),
                    'critic1_optimizer_state_dict': critic_optimizer1.state_dict(),
                    'critic2_optimizer_state_dict': critic_optimizer2.state_dict(),
                    'total_steps': total_steps,
                    'episode': episode,
                    'stage': current_stage,
                    'eval_reward': best_eval_reward,
                    'best_rewards': best_rewards
                }, checkpoint_path)
            
            break
        
        # Periodic checkpointing
        if episode % 250 == 0:
            replay_buffer.save('models/TD3/Replay_Buffer/mtp_replay_buffer.pkl')
            train_writer.flush()
            eval_writer.flush()
            checkpoint_path = f'models/TD3/Model/mtp_backup_{episode}.pth'
            torch.save({
                    'actor_state_dict': actor.state_dict(),
                    'critic1_state_dict': critic1.state_dict(),
                    'critic2_state_dict': critic2.state_dict(),
                    'actor_optimizer_state_dict': actor_optimizer.state_dict(),
                    'critic1_optimizer_state_dict': critic_optimizer1.state_dict(),
                    'critic2_optimizer_state_dict': critic_optimizer2.state_dict(),
                    'total_steps': total_steps,
                    'episode': episode,
                    'stage': current_stage,
                    'eval_reward': best_eval_reward,
                    'best_rewards': best_rewards
                }, checkpoint_path)
            print(f"Checkpoint saved at {checkpoint_path}")
    return actor, critic1, critic2, episode_rewards

In [64]:
env = CosysAirSimEnv_Velocity()

# Get environment dimensions
obs_dim = env.observation_space.shape[0]  
act_dim = env.action_space.shape[0]     
max_action = float(env.action_space.high[0]) 

# Run training
actor, critic1, critic2, rewards = run_training(
    env=env,
    obs_dim=obs_dim,
    act_dim=act_dim,
    max_action=max_action,
    episodes= 50_000
)

Connected!
Client Ver:3 (Min Req: 3), Server Ver:3 (Min Req: 3)

Using device: cuda
TensorBoard logs will be saved to: logs\TD3_train_20250127-195618


C:\Users\henhe\AppData\Local\Temp\ipykernel_13140\1026655346.py:75: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('models/TD3/Model/mtp_backup_2250.p

Loaded checkpoint successfully. Total steps: 149848
Loaded buffer successfully


Training:   0%|          | 1/50000 [00:06<85:16:26,  6.14s/it, stage=5, reward=-1.88, avg_reward=-1.88, success_count=0, min_dist=8.23]

Checkpoint saved at models/TD3/Model/mtp_backup_0.pth


Training:   0%|          | 5/50000 [03:58<663:34:11, 47.78s/it, stage=5, reward=-6.95, avg_reward=-4.52, success_count=0, min_dist=1.46] 


KeyboardInterrupt: 